In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel


#TODO: This file has problems

In [2]:
def contraction(params, x, p):
    # beta and x are kind of parameters. x is the empirical distribution of x?
    k = int(x.shape[1]/2)
    util1 = np.dot(x[:, 0:k], params[0:k]) + params[-2]*p[:,1]
    util2 = np.dot(x[:, k:2*k], params[k:2*k]) + params[-1]*p[:,0]
    contr_result = [np.exp(util1)/(1+np.exp(util1)),
                    np.exp(util2)/(1+np.exp(util2))]
    return np.array(contr_result).transpose()


def contraction_map(betas, x, p):
    """final result is beliefs of firm1/firm2"""
    for i in range(50):
        p = contraction(betas, x, p)
    return p

#actually caclualte an equilibrium in this game
N=1000
ps = np.random.randint(2, size=(N,2))
betas = np.array([1, 2, -2, -1])

#set up xs
xs = np.random.normal(scale=2, size=(N,2)) #assumes xs are independent? could use a copula?
xs = pd.DataFrame(xs,columns = ['x11','x12'])
xs = np.array(xs)


result_ps = contraction_map(betas,xs,ps)
us = np.random.logistic(size=(1000,2))

k = int(xs.shape[1]/2)
es = np.random.logistic(0, 1, (2,N) )
y1 = 1*(np.dot(xs[:, 0:k], betas[0:k])  + betas[-2]*result_ps[:,1] + es[0,:] >= 0)
y2 = 1*(np.dot(xs[:, k:2*k], betas[k:2*k]) + betas[-1]*result_ps[:,0] + es[1,:] >= 0)
ys = np.array([y1,y2]).transpose()

print(result_ps[(xs[:,1] > 2)].mean(axis=0))
print(ys[ (xs[:,1] > 2) ].mean(axis=0))

[0.22039895 0.99342023]
[0.24203822 0.98089172]


In [3]:
result_df = np.concatenate( (ys,xs ) ,axis=1)
result_df = pd.DataFrame(result_df, columns=['y1','y2','x11','x12'])
print(result_df)
result_df.to_csv('monte_carlo.csv')

      y1   y2       x11       x12
0    0.0  0.0 -1.602443 -0.906795
1    1.0  1.0  1.949413 -0.370789
2    1.0  0.0  1.308314 -0.680240
3    0.0  0.0 -2.743767 -2.448277
4    0.0  0.0  0.093243 -0.490898
..   ...  ...       ...       ...
995  0.0  0.0 -2.207663 -1.350822
996  0.0  0.0 -2.520320 -1.620732
997  0.0  1.0 -2.212792  0.949326
998  0.0  0.0 -0.501389 -1.285246
999  1.0  0.0  1.374238 -1.476167

[1000 rows x 4 columns]


In [4]:
class BayesNashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)

        p = self.endog
        p = contraction_map(params, self.exog, p)

        likelihood = contraction(params, self.exog, p)
        ll = self.endog*np.log(likelihood) + \
            (1-self.endog)*np.log(1-likelihood)
        return -1*ll.sum(axis=1)

        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        k = int(self.exog.shape[1]/2)
        x1 = np.concatenate( (self.exog[:, 0:k], self.endog[:,1].reshape(self.endog.shape[0],1) ) ,axis=1)
        x2 = np.concatenate( (self.exog[:, k:2*k], self.endog[:,0].reshape(self.endog.shape[0],1) ),axis=1)
        params1 =  sm.Logit(self.endog[:, 0], x1).fit().params
        params2 = sm.Logit(self.endog[:, 1], x2).fit().params
        start_params = np.concatenate((params1[0:-1],params2[0:-1],[params1[-1],params2[-1]] ))
        print(start_params)
        return super(BayesNashLogit, self).fit(start_params=start_params, **kwds)

        
        
N = result_df.shape[0]
ys = result_df[['y1','y2']]
xs = result_df[['x11' ,'x12']]
print(xs.shape)

model = BayesNashLogit(ys,xs)
model.loglikeobs(np.ones(xs.shape[1]+2))
model_fit = model.fit(xtol=1e-12,ftol=1e-12)
print(model_fit.summary(xname=["b11","d1","b12","d2"]))


(1000, 2)
Optimization terminated successfully.
         Current function value: 0.439798
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.279011
         Iterations 8
[ 0.9614755   1.97953106 -1.46233359 -0.43453159]
Optimization terminated successfully.
         Current function value: 0.698254
         Iterations: 174
         Function evaluations: 388
                            BayesNashLogit Results                            
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -698.25
Model:                 BayesNashLogit   AIC:                             1401.
Method:            Maximum Likelihood   BIC:                             1410.
Date:                Tue, 10 May 2022                                         
Time:                        08:56:39                                         
No. Observations:                1000                                         
Df Residuals:                     998      

In [5]:
#attempt at standard errors...
H = np.linalg.inv(model.hessian(model_fit.params)/N)
diag = np.diagonal(np.linalg.inv(model.hessian(model_fit.params)))
print(np.sqrt(-1*diag))

[0.06763441 0.12676757 0.16177376 0.22419225]


In [7]:
# Modify the variables
entry_data = pd.read_csv("../data/merged_entry.csv")
entry_data.loc[entry_data['HD'] > 0.0, 'HD_1'] = 1
entry_data.loc[entry_data['LO'] > 0.0, 'LO_1'] = 1
entry_data['income_2'] = np.log(1 + entry_data['income_per_capita'])
entry_data['population_2'] = np.log(1 + entry_data['population'])
entry_data['under44_1_2'] = np.log(1 + entry_data['under44_1'])
entry_data['under44_2_2'] = np.log(1 + entry_data['under44_2'])
entry_data['under44_3_2'] = np.log(1 + entry_data['under44_3'])
entry_data['older65_1_2'] = np.log(1 + entry_data['older65_1'])
entry_data['older65_2_2'] = np.log(1 + entry_data['older_65_2'])
entry_data = entry_data.fillna(0)

/home/erichschulman/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
# Fit the model
N = entry_data.shape[0]
x = entry_data[['income_2','population_2','income_2','population_2']].copy()
y = entry_data[['HD_1','LO_1']]

model = BayesNashLogit(y,x).fit(xtol=1e-6,ftol=1e-6)
model.summary()

Optimization terminated successfully.
         Current function value: 0.416220
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.520699
         Iterations 7
[-0.14254449  0.50316223 -0.13063233  0.42585139 -3.37383234 -3.33023913]


/home/erichschulman/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            BayesNashLogit Results                            
==============================================================================
Dep. Variable:       ['HD_1', 'LO_1']   Log-Likelihood:                -2676.3
Model:                 BayesNashLogit   AIC:                             5357.
Method:            Maximum Likelihood   BIC:                             5368.
Date:                Tue, 10 May 2022                                         
Time:                        08:56:52                                         
No. Observations:                2155                                         
Df Residuals:                    2153                                         
Df Model:                           1                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
income_2        -0.1636      0.038     -4.280      0.000      -0.239      -0.089
population_2     0.2182      0.042      5.207      0.000       0.136       0.300
income_2        -0.1831      0.034     -5.339      0.000      -0.250      -0.116
population_2     0.2013      0.036      5.534      0.000       0.130       0.273
par0             1.0537      0.282      3.732      0.000       0.500       1.607
par1             0.1174      0.216      0.543      0.587      -0.306       0.541
================================================================================
"""